In [36]:
import pandas as pd

## List of projects

One of parameters / input data is Pandas DataFrame with projects as one of main columns.

This DataFrame will be enriched by this notebook / script with project metadata features.

In [81]:
infile='df_commits_with_one_project.parquet'
infile='../data/combined_df_filename'
infile

'../data/combined_df_filename'

In [82]:
pwd()

'/home/jnareb/echoes_of_vulnerabilities_code/projects_stats'

In [83]:
df=pd.read_parquet(infile)

Examine the DataFrame

In [84]:
df.columns

Index(['commit', 'commit_cves', 'commiter_time', 'author_time',
       'project_names', 'total_number_of_files', 'dep_requirements',
       'dep_cargo', 'dep_go_mod', 'dep_ivy',
       ...
       'lang_AspectJ', 'lang_HLSL', 'lang_ShaderLab', 'lang_SMT', 'lang_XC',
       'lang_Charity', 'lang_xBase', 'lang_WebIDL', 'lang_Genie', 'lang_Gosu'],
      dtype='object', length=9339)

In [102]:
#projects=df['project_name'].unique()
projects=df['project_names'].unique()
len(projects)

[p for pp.split(';') for pp in projects if (pp is not None)]

NameError: name 'pp' is not defined

The starting point could be just a list of projects

In [13]:
projects_s=pd.Series(projects)

In [12]:
pd.Series(projects).to_csv('interesting_projects_list.csv', sep=';', header=False, index=False)

## Retrieving the list of projects (alt)

In [45]:
project_s=pd.read_csv('interesting_projects_list.csv', sep=';', header=None)[0]
project_s

0                                          mirror-rpm_exim
1                                      buildroot_buildroot
2                                                Exim_exim
3                                    pfsense_FreeBSD-ports
4        salsa.debian.org_security-tracker-team_securit...
                               ...                        
35753                            ESIEASecurityClub_exploit
35754                                       klange_toaruos
35755                                LearnPress_learnpress
35756                         apache_cordova-plugin-camera
35757                                 stong_CVE-2020-15368
Name: 0, Length: 35758, dtype: object

In [47]:
projects=project_s.unique()
projects[:10]

array(['mirror-rpm_exim', 'buildroot_buildroot', 'Exim_exim',
       'pfsense_FreeBSD-ports',
       'salsa.debian.org_security-tracker-team_security-tracker',
       'git-portage_git-portage', 'felenk_ports', 'gentoo_gentoo',
       'ulm_gentoo-x86-historical', 'bitrig_bitrig-ports'], dtype=object)

## Accessing MongoDB database

Accessing the World of Code MongoDB database with various metadata is described in https://github.com/woc-hack/tutorial#mongo-database

> On the da1 server, there is a MongoDB server holding some relevant data. This data includes some information that was used for data analysis in the past. Mongo provides an excellent place to store relatively small data without requiring relational information.
>
> Two collections the WoC database cand be helpful for sampling projects and authors `A_metadata.V` and `P.metadata.V` where `V` represents the version (e.g., `T`) , `A` stands for aliased author id and `P` for deforked repository name.

There is no direct access to the `da1` server.  One needs to tunnel connection via `da0`, otherwise one would get the following error

```
ServerSelectionTimeoutError: da1.eecs.utk.edu:27017: [Errno 113] No route to host, Timeout: 30s, Topology Description: <TopologyDescription id: 639216df3fa72534c7d7e1c5, topology_type: Unknown, servers: [<ServerDescription ('da1.eecs.utk.edu', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('da1.eecs.utk.edu:27017: [Errno 113] No route to host')>]>
```

One possible solution is to use [`ssh-pymongo`](https://pypi.org/project/ssh-pymongo/) (which in turn is a thin wrapper around [`sshtunnel`](https://pypi.org/project/sshtunnel/) and [`pymongo`](https://pypi.org/project/pymongo/)), see e.g.  
https://stackoverflow.com/questions/56239184/python-script-to-connect-to-remote-mongodb-using-ssh-tunnel-and-pymongo-client-i

The configuration for connecting to World of Code servers via `da0` gateway looks like this:
```
Host da0
	Hostname da0.eecs.utk.edu
	Port 443
	User <username>
	IdentityFile <path to private key, e.g. %d/.ssh/id_rsa_woc>
```

> When on the da1 server, you can gain access to the MongoDB server simply by running the command `mongo`, or, when on any other da server, you can gain access by running `mongo --host "da1.eecs.utk.edu"`.

from https://github.com/woc-hack/tutorial

This means that when being logged onto WoC gateway, namely 'da0' host, one would need to use the latter

In [48]:
import os

username = os.environ['JUPYTERHUB_USER']
username

'jnareb'

In [49]:
home = os.getenv('HOME', '/home/'+username)
home

'/home/jnareb'

This SSH configuration, reproduced as `ssh_pymongo` configuration, looks like this (together with information about the host where MongoDB database is)

**NOTE**: since 12.12.2022, after recovered from da2 failure,

> 3. if you use mongodb, change server to da5 from da1

source: Discord (https://discord.gg/fKPFxzWqZX)

In [50]:
from ssh_pymongo import MongoSession

session = MongoSession(
    host="da0.eecs.utk.edu",
    port=443,
    user=username,                      # this may need adjusting
    key=home+'/.ssh/id_rsa_woc',        # this may need adjusting
    uri="mongodb://da5.eecs.utk.edu/"
)

client=session.connection
client

MongoClient(host=['127.0.0.1:45255'], document_class=dict, tz_aware=False, connect=True)

Example from [PyMongo documentation](https://pymongo.readthedocs.io/en/stable/tutorial.html#getting-a-single-document-with-find-one): 

In [57]:
import pprint
pprint.pprint(client['WoC']['A_metadata.U'].find_one())

{'AuthorID': 'Sulthan-Basha <34763305+Sulthan-Basha@users.noreply.github.com>',
 'EarliestCommitDate': 1619434588,
 'FileInfo': {'other': 4},
 'Gender': 'male',
 'LatestCommitDate': 1619434588,
 'MonNcmt': {'2018-10': 2,
             '2019-07': 2,
             '2019-08': 1,
             '2019-09': 4,
             '2020-05': 2,
             '2020-06': 2,
             '2020-10': 2,
             '2021-04': 4,
             '2021-07': 3},
 'MonNprj': {'2018-10': 1,
             '2019-07': 1,
             '2019-08': 1,
             '2019-09': 2,
             '2020-05': 1,
             '2020-06': 1,
             '2020-10': 1,
             '2021-04': 1,
             '2021-07': 1},
 'NumActiveMon': 9,
 'NumCommits': 22,
 'NumFiles': 4,
 'NumOriginatingBlobs': 10,
 'NumProjects': 10,
 '_id': ObjectId('628f66290788e585aa1d4d36')}


Example from the WoC tutorial, split into two, slightly modified:

In [51]:
#import pymongo
#
#client = pymongo.MongoClient("mongodb://da1.eecs.utk.edu/")

In [61]:
db = client['WoC']
coll = db['A_metadata.U']

dataset = coll.find({}, no_cursor_timeout=True).limit(10)
for data in dataset:
    a = data["AuthorID"].encode('utf-8').strip()
    print(a)

dataset.close()

b'Sulthan-Basha <34763305+Sulthan-Basha@users.noreply.github.com>'
b'a <250550879@qq.com>'
b'Christopher Hannon <nc@dhcp93.merunorth110.iit.edu>'
b'Emily Hedgecock <Emily@omsystems.co.uk>'
b'tb-lab <f297138735+tb@gmail.com>'
b'itsamemonisha <77173644+itsamemonisha@users.noreply.github.com>'
b'fredericnasri <nasrifreddy@gmail.com>'
b'yukasansan <64906781+yukasansan@users.noreply.github.com>'
b'xParanormalx <info@2moons.eu>'
b'ibrahem-abualfoul <76413554+ibrahem-abualfoul@users.noreply.github.com>'


## Browsing project metadata

Let's take an example project

In [63]:
project=projects_s[1]
project

'buildroot_buildroot'

Find every metadata about this project in MongoDB

In [65]:
coll = db['P_metadata.U']
coll

Collection(Database(MongoClient(host=['127.0.0.1:45255'], document_class=dict, tz_aware=False, connect=True), 'WoC'), 'P_metadata.U')

Let's examine the basic structure of the MonngoDB collection, based on an example of a single project

In [66]:
pprint.pprint(coll.find_one())

{'CommunitySize': 1,
 'Core': {'johan-84 <83859842+johan-84@users.noreply.github.com>': '2'},
 'EarliestCommitDate': 1620460902,
 'FileInfo': {'other': 2},
 'LatestCommitDate': 1620781111,
 'MonNauth': {'2021-05': 1},
 'MonNcmt': {'2021-05': 2},
 'NumActiveMon': 1,
 'NumAuthors': 1,
 'NumBlobs': 2,
 'NumCommits': 2,
 'NumCore': 1,
 'NumFiles': 2,
 'NumForks': 0,
 'ProjectID': 'johan-84_johan-84',
 '_id': ObjectId('628e3343f7956706ea414615')}


Let's examine selected project

In [69]:
project

'buildroot_buildroot'

In [71]:
pprint.pprint(coll.find_one({'ProjectID': project}))

{'ApiInfo': {'C:': 37203,
             'C:alloca.h': 655,
             'C:assert.h': 3231,
             'C:cdefs.h': 552,
             'C:ctype.h': 4627,
             'C:dirent.h': 1736,
             'C:elf.h': 898,
             'C:endian.h': 957,
             'C:errno.h': 14651,
             'C:fcntl.h': 6722,
             'C:features.h': 11407,
             'C:file.h': 550,
             'C:float.h': 696,
             'C:getopt.h': 1825,
             'C:grp.h': 895,
             'C:if.h': 434,
             'C:in.h': 1688,
             'C:inet.h': 1136,
             'C:inttypes.h': 604,
             'C:ioctl.h': 2152,
             'C:langinfo.h': 481,
             'C:libgen.h': 709,
             'C:libmtd.h': 445,
             'C:libubi.h': 418,
             'C:limits.h': 4569,
             'C:link.h': 697,
             'C:locale.h': 1948,
             'C:malloc.h': 550,
             'C:mman.h': 2270,
             'C:mount.h': 657,
             'C:mtd-user.h': 585,
             'C:name

Let's select simple metadata

In [72]:
pprint.pprint(coll.find_one({'ProjectID': project},
                            {'_id': 0, 'CommunitySize': 1, 'NumCore': 1,
                             'EarliestCommitDate': 1, 'LatestCommitDate': 1,
                             'NumActiveMon': 1,
                             'NumAuthors': 1,
                             'NumBlobs': 1,
                             'NumCommits': 1,
                             'NumFiles': 1,
                             'NumForks': 1,
                             'NumStars': 1,
                             'ProjectID': 1,
                             'RootFork': 1,
                            }))

{'CommunitySize': 4897,
 'EarliestCommitDate': 958277795,
 'LatestCommitDate': 1637721166,
 'NumActiveMon': 259,
 'NumAuthors': 4347,
 'NumBlobs': 694969,
 'NumCommits': 375883,
 'NumCore': 239,
 'NumFiles': 604548,
 'NumForks': 4390,
 'NumStars': 12124,
 'ProjectID': 'buildroot_buildroot',
 'RootFork': 'buildroot_buildroot'}


In [77]:
project='salsa.debian.org_security-tracker-team_security-tracker'
pprint.pprint(coll.find_one({'ProjectID': project}))

{'ApiInfo': {'PY:': 1144,
             'PY:BaseHTTPServer': 41,
             'PY:F401': 6,
             'PY:SOAPpy': 12,
             'PY:SocketServer': 41,
             'PY:__future__.print_function': 135,
             'PY:__future__.with_statement': 43,
             'PY:apsw': 247,
             'PY:apt_pkg': 100,
             'PY:apt_pkg.version_compare': 105,
             'PY:argparse': 80,
             'PY:base64': 200,
             'PY:bin.tracker_data.TrackerData': 3,
             'PY:bugs': 517,
             'PY:bz2': 25,
             'PY:cPickle': 213,
             'PY:cStringIO': 265,
             'PY:cStringIO.StringIO': 41,
             'PY:cgi': 64,
             'PY:cjson': 16,
             'PY:collections': 61,
             'PY:collections.OrderedDict': 6,
             'PY:collections.namedtuple': 135,
             'PY:config': 128,
             'PY:datetime': 34,
             'PY:datetime.datetime': 31,
             'PY:dateutil.relativedelta': 18,
             'PY:debian

## Extracting projects' metadata

Let's do this for a few example projects, and create dataframe

In [73]:
from tqdm.notebook import trange, tqdm

In [76]:
projects_info=[]

for project in tqdm(projects[:10]):
    info = coll.find_one({'ProjectID': project},
                            {'_id': 0, 'CommunitySize': 1, 'NumCore': 1,
                             'EarliestCommitDate': 1, 'LatestCommitDate': 1,
                             'NumActiveMon': 1,
                             'NumAuthors': 1,
                             'NumBlobs': 1,
                             'NumCommits': 1,
                             'NumFiles': 1,
                             'NumForks': 1,
                             'NumStars': 1,
                             'ProjectID': 1,
                             'RootFork': 1,
                            })
    projects_info.append(info)
    
projects_info_df = pd.DataFrame.from_records(projects_info, index='ProjectID')
projects_info_df

  0%|          | 0/10 [00:00<?, ?it/s]

,NumAuthors,NumActiveMon,EarliestCommitDate,NumBlobs,LatestCommitDate,NumCore,NumFiles,CommunitySize,NumCommits,NumForks,RootFork,NumStars
ProjectID,,,,,,,,,,,,
mirror-rpm_exim,35,111,1113883342,396,1626906630,6,159,1,224,0,NaN,NaN
buildroot_buildroot,4347,259,958277795,694969,1637721166,239,604548,4897,375883,4390,buildroot_buildroot,12124.0
Exim_exim,127,200,1097062611,49948,1636918288,6,7455,130,8456,132,Exim_exim,542.0
pfsense_FreeBSD-ports,2520,329,769769985,2951925,1637330126,210,1277004,1184,4659656,1340,freebsd_freebsd-ports,1483.0
salsa.debian.org_security-tracker-team_security-tracker,228,206,1098910404,185522,1637978885,13,544,27,207994,0,NaN,NaN
git-portage_git-portage,457,182,964744542,3494204,1439152874,107,566640,5,2212980,3,git-portage_git-portage,6.0
felenk_ports,321,294,833842030,644444,1631646578,32,134361,61,488853,78,openbsd_ports,324.0
gentoo_gentoo,2600,257,964744542,18381095,1635933891,156,1142501,1268,5303764,1651,gentoo_gentoo,1648.0
ulm_gentoo-x86-historical,668,182,964744542,3406841,1439152874,125,566553,1,1688485,0,NaN,NaN
